SVMとは何か

どのようなときに使うのか

答え：分類にも回帰にも使用可能。２クラス分類だけでなく多クラス分類も可能。また線形だけでなく非線形な分類も可能。


どのような仕組みなのか

分類：サポートベクター（最も近い点）のマージン（サンプルが含まれない距離）を最大化するような境界線を引く。サポートベクター以外の点はこの境界線は関係ない。コストペナルティとスラック変数を用いた誤分類されている度合いと、マージンの広さを計算し、その合計が最も小さくなる線の引き方をする。

回帰：ある距離のイプシロンチューブの中にサンプルを含むようにする。その中では誤差がないものとして扱う。その他のサンプルに対してはイプシロンチューブから遠いほど一次関数的に増加するペナルティを課す。

どういった利点欠点があるのか

利点：カーネルを使用することで非線形な分類が可能。特徴量が増えても精度が高い。

欠点：

・１対他(one-versus-the-rest,ovr)の場合、各SVMにおいて２つのクラスのバランスが悪い。また、各SVMによる予測に差が出てそれぞれの優劣を決められない。

・１対１(one-versus-one,ovo)の場合、時間がかかる。n*(n-1)/2の回数のSVMを行う必要がある。


In [1]:
#必要なライブラリをimport
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

In [2]:
#データを読み込むコードを記述
df = pd.read_csv('agaricus-lepiota.data.txt',header=None)
df.columns = ['target','1. cap-shape','2. cap-surface','3. cap-color','4. bruises?','5. odor','6. gill-attachment',
             '7. gill-spacing','8. gill-size','9. gill-color','10. stalk-shape','11. stalk-root',
             '12. stalk-surface-above-ring','13. stalk-surface-below-ring','14. stalk-color-above-ring',
             '15. stalk-color-below-ring','16. veil-type','17. veil-color','18. ring-number',
             '19. ring-type','20. spore-print-color','21. population','22. habitat:']

In [3]:
#データを眺める
df.head(10)

,target,1. cap-shape,2. cap-surface,3. cap-color,4. bruises?,5. odor,6. gill-attachment,7. gill-spacing,8. gill-size,9. gill-color,...,13. stalk-surface-below-ring,14. stalk-color-above-ring,15. stalk-color-below-ring,16. veil-type,17. veil-color,18. ring-number,19. ring-type,20. spore-print-color,21. population,22. habitat:
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


In [4]:
#文字列から整数へ変換するコードを記述
from sklearn import preprocessing

def encode_features(df):
    features = ['target','1. cap-shape','2. cap-surface','3. cap-color','4. bruises?','5. odor','6. gill-attachment',
             '7. gill-spacing','8. gill-size','9. gill-color','10. stalk-shape','11. stalk-root',
             '12. stalk-surface-above-ring','13. stalk-surface-below-ring','14. stalk-color-above-ring',
             '15. stalk-color-below-ring','16. veil-type','17. veil-color','18. ring-number',
             '19. ring-type','20. spore-print-color','21. population','22. habitat:']
    
    for feature in features:
        le = preprocessing.LabelEncoder()#データを数値で表現する
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

encode_features(df).sample(3)

,target,1. cap-shape,2. cap-surface,3. cap-color,4. bruises?,5. odor,6. gill-attachment,7. gill-spacing,8. gill-size,9. gill-color,...,13. stalk-surface-below-ring,14. stalk-color-above-ring,15. stalk-color-below-ring,16. veil-type,17. veil-color,18. ring-number,19. ring-type,20. spore-print-color,21. population,22. habitat:
1230,0,5,0,4,0,5,1,1,0,5,...,0,7,7,0,2,1,0,3,3,1
6542,1,2,3,4,0,2,1,0,1,0,...,1,7,6,0,2,1,0,7,4,2
673,0,5,0,9,1,3,1,1,1,10,...,2,7,7,0,2,1,4,6,4,0


In [5]:
#データセットを分割するコードの記述
X = df.drop('target',axis=1)
y = df['target']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=0)

In [20]:
#標準化を行うコードを記述
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
#fit_transform：パラメータ計算とデータ変換をまとめて実行
X_test_std = stdsc.transform(X_test)
#transform:パラメータを元にデータ変換
#fit:パラメータ計算(mean,std)

In [21]:
X_test_std

array([[ 1.00847269,  0.94758635, -0.25107982, ..., -0.65618438,
        -0.55823423,  2.05731692],
       [-0.85094651,  0.94758635, -0.63382345, ..., -0.65618438,
         1.09560464, -0.87886786],
       [ 1.00847269,  0.94758635,  1.66263832, ..., -0.23555337,
        -1.38515366,  0.88284301],
       ..., 
       [-0.85094651,  0.94758635, -1.01656708, ...,  1.44697068,
         0.26868521,  1.47007996],
       [-2.09055931,  0.12590662,  1.2798947 , ..., -0.65618438,
        -1.38515366,  0.88284301],
       [ 1.00847269,  0.94758635, -0.25107982, ..., -0.23555337,
         0.26868521, -0.87886786]])

なぜSVMを行う際に標準化が必要なのか

答え：各特徴量間でスケールの違いがあるとモデルをうまく学習させることができないことがあるため。勾配降下法のパラメータ更新時に、各パラメータ更新幅に偏りが生じる。

カーネルとは何か、またどう使い分けるのが良いか。

答え：非線形問題の場合、高次元空間へ移し分類するが、その際に各点の内積に対し計算を行うのではなく、関数を用意し計算を簡単に済ませられるようにする。この関数のことをカーネルという。

線形カーネルは、線形分離可能な場合。テキストデータ分類時によく用いられる。

多項式カーネルは、画像分類の際によく用いられる。

RBFカーネルは、最も汎用的なカーネル。非線形分類に用いられる。

シグモイドカーネルは、ニューラルネットワークの代わりに用いられる。

コストペナルティCとは何か

答え：マージンより外の点にペナルティを与える際の、ペナルティの度合いの大きさ。


In [8]:
#グリッドサーチのコードを記述
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

#最適化したいパラメータの設定
tuned_parameters = [
    {'C': [1,10,100,1000], 'kernel': ['linear']},
    {'C': [1,10,100,1000], 'kernel': ['rbf'], 'gamma': [0.001,0.0001]},
    {'C': [1,10,100,1000], 'kernel': ['poly'], 'degree': [2,3,4], 'gamma': [0.001,0.0001]},
    {'C': [1,10,100,1000], 'kernel': ['sigmoid'], 'gamma': [0.001,0.0001]}
]

#パラメータを最適化
score = 'f1'
clf = GridSearchCV(
SVC(),#識別器
tuned_parameters,#最適化したいパラメータセット
cv=5,#交差検証の回数
scoring='%s_weighted' % score )#モデルの評価関数の指定

#最適化実行
clf.fit(X_train,y_train)

/Users/sakashitatakuji/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/sakashitatakuji/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/sakashitatakuji/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']}, {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.001, 0.0001]}, {'C': [1, 10, 100, 1000], 'kernel': ['poly'], 'degree': [2, 3, 4], 'gamma': [0.001, 0.0001]}, {'C': [1, 10, 100, 1000], 'kernel': ['sigmoid'], 'gamma': [0.001, 0.0001]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='f1_weighted',
       verbose=0)

In [9]:
#各試行のスコア確認
clf.grid_scores_

[mean: 0.98307, std: 0.00945, params: {'C': 1, 'kernel': 'linear'},
 mean: 0.98462, std: 0.00416, params: {'C': 10, 'kernel': 'linear'},
 mean: 0.98462, std: 0.00416, params: {'C': 100, 'kernel': 'linear'},
 mean: 0.98462, std: 0.00416, params: {'C': 1000, 'kernel': 'linear'},
 mean: 0.94767, std: 0.00375, params: {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'},
 mean: 0.88665, std: 0.00438, params: {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'},
 mean: 0.95476, std: 0.00431, params: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'},
 mean: 0.94674, std: 0.00386, params: {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'},
 mean: 0.98892, std: 0.00246, params: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'},
 mean: 0.94982, std: 0.00524, params: {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'},
 mean: 1.00000, std: 0.00000, params: {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'},
 mean: 0.97014, std: 0.00503, params: {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'},
 mean: 0.35170, std: 0.00022, params: {'C': 1, 'degree': 2

In [10]:
#最適化したパラメータ確認
clf.best_params_

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

In [11]:
#SVMの学習およびテストをするコードを記述
rbf_svm = SVC(kernel='rbf', gamma=0.001, C=1000, random_state=0)
rbf_svm.fit(X_train,y_train)

from sklearn.metrics import accuracy_score
predictions = rbf_svm.predict(X_test)#X_testを使って予測する
print(accuracy_score(y_test,predictions))

1.0
